In [14]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [15]:
filename='ReportHistory-4000009439.xlsx'

In [16]:
def load_xls_report(filename):
    data = pd.read_excel(filename, header=None)
    start_index = data[data.iloc[:, 0]=='Transacciones'].index[0]
    end_index = data.iloc[start_index:, 0].isna().idxmax()
    columns = data.iloc[start_index+1].tolist()
    data.columns = columns
    balance = data['Beneficio'].iloc[start_index+2]
    data = data.iloc[start_index+3:end_index-1,:]
    resultados = []
    for i in range(0, len(data) - 1, 2):
        fila_in = data.iloc[i]
        fila_out = data.iloc[i + 1]
        if fila_in['Dirección'] == 'in' and fila_out['Dirección'] == 'out':
            precio_entrada = fila_in['Precio']
            precio_salida = fila_out['Precio']
            beneficio = fila_out['Beneficio'] - abs(fila_in['Comisión']) - abs(fila_out['Comisión'])
            balance += beneficio
            resultados.append({
                'Fecha': fila_in['Fecha/Hora'],
                'Símbolo': fila_in['Símbolo'],
                'Precio Entrada': precio_entrada,
                'Precio Salida': precio_salida,
                'Volumen': fila_in['Volumen '],
                'Beneficio': beneficio,
                'Balance': balance
            })
    return pd.DataFrame(resultados)

In [17]:
# Statistics
df = load_xls_report(filename)
total_trades = df.shape[0]
profit_trades = df[df['Beneficio']>0].shape[0]
profit_avg = df[df['Beneficio']>0]['Beneficio'].mean()
loss_trades = df[df['Beneficio']<0].shape[0]
loss_avg = df[df['Beneficio']<0]['Beneficio'].mean()
beneficio_neto  = (profit_avg*profit_trades)+(loss_avg*loss_trades)
total_trades = profit_trades + loss_trades
expected_profit = beneficio_neto / total_trades
profit_avg_points = abs(df[df['Beneficio']>0]['Precio Entrada'] - df[df['Beneficio']>0]['Precio Salida']).mean()
loss_avg_points = abs(df[df['Beneficio']<0]['Precio Entrada'] - df[df['Beneficio']<0]['Precio Salida']).mean()
punto_equilibrio = abs(loss_avg)/(abs(loss_avg)+profit_avg)
win_rates = profit_trades/total_trades
balances = df['Balance']
cummax_balance = balances.cummax()
drawdowns = cummax_balance - balances
drawdown_percentages = drawdowns / cummax_balance
max_drawdown_percentage = drawdown_percentages.max() * 100

In [18]:
print(f"Número de operaciones totales: {total_trades}")
print(f"Número de operaciones ganadoras: {profit_trades}")
print(f"Número de operaciones perdedoras: {loss_trades}")
print(f"Media de puntos en operaciones ganadoras {profit_avg_points:.2f} puntos")
print(f"Media de puntos en operaciones perdedoras {loss_avg_points:.2f} puntos")
print(f"Ganancia media: {profit_avg:.2f}€")
print(f"Pérdida media: {loss_avg:.2f}€")
print(f"Rentabilidad esperada: {expected_profit:.4f}€")
print(f"Porcentaje de aciertos actual: {win_rates*100.0:.2f}%")
print(f"Porcentaje de aciertos para equilibrio: {punto_equilibrio*100.0:.2f}%")
print(f"El máximo drawdown es: {max_drawdown_percentage:.2f}%")

Número de operaciones totales: 112
Número de operaciones ganadoras: 69
Número de operaciones perdedoras: 43
Media de puntos en operaciones ganadoras 2040.69 puntos
Media de puntos en operaciones perdedoras 2073.70 puntos
Ganancia media: 7.48€
Pérdida media: -6.24€
Rentabilidad esperada: 2.2129€
Porcentaje de aciertos actual: 61.61%
Porcentaje de aciertos para equilibrio: 45.47%
El máximo drawdown es: 3.75%


In [19]:
df

,Fecha,Símbolo,Precio Entrada,Precio Salida,Volumen,Beneficio,Balance
0,2024.09.26 10:30:36,GDAXI,19101.6,19118.9,0.26,43.54,3647.00
1,2024.09.26 12:09:21,GDAXI,19134.6,19151.9,0.26,43.54,3690.54
2,2024.09.26 12:36:00,GDAXI,19142.6,19145.9,0.26,7.14,3697.68
3,2024.09.26 13:08:36,GDAXI,19119.6,19116.9,0.26,-8.46,3689.22
4,2024.09.26 15:10:35,GDAXI,19131.6,19139.9,0.17,13.17,3702.39
...,...,...,...,...,...,...,...
107,2024.10.03 21:15:38,NDX,19760.7,5695.5,0.01,0.24,3849.69
108,2024.10.03 21:15:58,SP500,5696.7,19770.8,0.01,0.90,3850.59
109,2024.10.03 21:16:57,GDAXI,19005.2,5696.3,0.01,0.01,3850.60
110,2024.10.03 21:55:46,GDAXI,19001.7,5695.3,0.01,0.12,3850.72
